In [18]:
!git clone https://github.com/samoletpanfilov/reinforcement_task

!mv reinforcement_task/data data/

!rm -r reinforcement_task
!ls

Cloning into 'reinforcement_task'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 3), reused 18 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), 278.35 KiB | 3.16 MiB/s, done.
data  result.xlsx  sample_data


In [19]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [20]:
train = pd.read_excel("data/train.xlsx").set_index("dt")
test = pd.read_excel("data/test.xlsx").set_index("dt")

In [21]:
import statsmodels.api as sm
def predict(data, step):
    mod = sm.tsa.statespace.SARIMAX(data,
                                    order=(0,2,1),
                                    seasonal_order=(1,1,2,12),
                                    freq="W-MON",
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    return results.forecast(steps=step)

In [22]:
choices = []
step = 5
for i in range(0, len(test)):
    data = pd.concat([train['Цена на арматуру'], test['Цена на арматуру'][:i]])
    fore = predict(data, step)
    # n = 0
#     for k in fore:
#         n += 1
#         if k <= data[-1]:
#             break
#     choices.append(n if sum(choices) - i - 1 < 1 else 0)

    # if data[-1] < np.min(fore):
    #     choices.append(step if sum(choices) - i < 1 else 0)
    # else:
    #     choices.append(1 if sum(choices) - i  < 1 else 0)

    if data[-1] < np.min(fore):
        choices.append(max(0, step - (sum(choices) - i)))
    else:
        choices.append(max(0, 1 - (sum(choices) - i)))



/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  

In [23]:
len(choices)

28

In [24]:
def decision_prices(test):

    test = test.set_index('dt')
    tender_price = test['Цена на арматуру']
    decision = test['Объем']
    start_date = test.index.min()
    end_date = test.index.max()

    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost # Возвращаем затраты на периоде

In [25]:
final = test.reset_index()
final['Объем'] = choices
decision_prices(final)

1205500

In [28]:
final.to_excel("data/test.xlsx", index=False)

In [29]:
# Download
from google.colab import files
files.download('data/test.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>